In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),                      # [0,255] → [0,1]
    transforms.Normalize((0.5,), (0.5,))        # mean, std
])


In [3]:
train_dataset = datasets.FashionMNIST(
    root="./dataset",
    train=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root="./dataset",
    train=False,
    transform=transform
)


In [ ]:
batch_size = 64

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)


In [5]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 28x28 → 28x28
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → 14x14

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # → 14x14
            nn.ReLU(),
            nn.MaxPool2d(2)                              # → 7x7
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FashionCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


In [8]:
epochs = 10

for epoch in range(epochs):
    loss = train(model, train_loader, criterion, optimizer)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss:.4f}")


Epoch [1/10], Loss: 0.5612
Epoch [2/10], Loss: 0.3676
Epoch [3/10], Loss: 0.3156
Epoch [4/10], Loss: 0.2809
Epoch [5/10], Loss: 0.2554
Epoch [6/10], Loss: 0.2354
Epoch [7/10], Loss: 0.2176
Epoch [8/10], Loss: 0.2059
Epoch [9/10], Loss: 0.1923
Epoch [10/10], Loss: 0.1804


In [9]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total


In [10]:
accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 91.53%


In [11]:
classes = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

def predict(model, image):
    model.eval()
    image = image.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)

    return classes[pred.item()]


In [12]:
image, label = test_dataset[0]
print("Prediction:", predict(model, image))
print("Ground Truth:", classes[label])


Prediction: Ankle boot
Ground Truth: Ankle boot


# Fashion-MNIST CNN Pipeline (End-to-End)

## Workflow

- Raw Images  
- Transforms (ToTensor, Normalize)  
- DataLoader (Batching, Shuffling)  
- CNN Feature Extractor (Conv → ReLU → Pool)  
- Fully Connected Classifier (Flatten → Dense → Dropout → Output)  
- CrossEntropy Loss  
- Adam Optimizer  
- Training Loop (Forward → Loss → Backprop → Update)  
- Evaluation / Inference  

---

## Explanation

- **Raw Images**  
  28×28 grayscale images from the Fashion-MNIST dataset

- **Transforms**  
  Convert images to tensors and normalize pixel values

- **DataLoader**  
  Handles batching, shuffling, and efficient data loading

- **CNN Feature Extractor**  
  Uses convolution and pooling layers to learn spatial features

- **Classifier**  
  Fully connected layers map features to 10 clothing classes

- **Loss Function**  
  CrossEntropyLoss measures classification error

- **Optimizer**  
  Adam optimizer updates network weights

- **Training Loop**  
  Forward pass → loss computation → backpropagation → weight update

- **Evaluation / Inference**  
  Computes accuracy and predicts unseen images
